In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
def get_rotations_csv(file_path, result_path, number):
"""
Функция для обработки данных с поворотных осей. Сохраняет данные в массив .npy
"""
    df =  pd.read_csv(file_path, sep=';', header=0, decimal=',') 
    data = df.to_numpy()
    angle = data[:,1]
    for i in range(0, len(angle)-1):
        if abs(angle[i] - angle[i+1])>1000:
            if (angle[i+1] - angle[i])>0:
                angle[i+1:] -= 11520
            else:
                angle[i+1:] += 11520
    if data.shape[1] == 4:
        data = np.delete(data, 3, axis=1)
    np.save(f"{result_path}/data{number}.npy", data)

In [3]:
def get_rotations_txt(file_path, result_path, number):
"""
Функция для обработки  с поворотных осей. Сохраняет данные в массив .npy
"""
    data = np.genfromtxt(file_path)
    np.save(f"{result_path}/data{number}.npy", data)

In [4]:
def filter(data): # фильтрация выбросов 
"""
Функция для удаления строк из данных блока, где происходит явный скачок производной по угловой скорости 
(разница углов более 5000 град/ч)
"""
    outliers = []
    for col in [1,2,3]:
        omega = data[:, col] * 3600 * 180 * 200 / np.pi
        for i in range(1, omega.size-1):
            if (abs(omega[i]-omega[i-1])>5000 and abs(omega[i]-omega[i+1])) > 5000:
                outliers.append(i)
    filtered_data = np.delete(data, outliers, axis=0)
    
    return filtered_data

In [5]:
def get_data_ibl(file_path, result_path):
"""
Функция для обработки  данных с блока. Сохраняет данные в массив .npy.
Данные с блока должны иметь вид:
[time, dfiX, dfiY, dfiZ, wX, wY, wZ, TaX, TaY, TaZ, TlgX, TlgY, TlgZ, Text]
или
[time, dfiX, dfiY, dfiZ, wX, wY, wZ, TlgY, TlgZ, Text]
если вместо термодатчиков акселерометров используются гироскопные.
"""
    data = np.loadtxt(file_path, skiprows=1)
    filtered_data = filter(data)
    np.save(f"{result_path}/data_ibl.npy", filtered_data)

In [7]:
"""
ВХОДНЫЕ ДАННЫЕ
"""
path = "20250502"
rot = "csv" # "txt"
path_1 = f"{path}/{path}_1.csv"
path_2 = f"{path}/{path}_2.csv"
path_ibl = f"{path}/short.txt"

In [8]:
if rot == "csv":
    get_rotations_csv(path_1, path, 1)
    get_rotations_csv(path_2, path, 2)
if rot == "txt":
    get_rotations_txt(path_1, path, 1)
    get_rotations_txt(path_2, path, 2)

In [15]:
get_data_ibl(path_ibl, path)